In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [32]:
# importing the csv file, do basic inspection, dropping NaN values

In [33]:
# depression by level of education 2014 dataset
df_edu = pd.read_csv('data/depression-by-level-of-education-employment.csv')
df_gen = pd.read_csv('data/gender/gender_hdi.csv')
df_country = pd.read_csv('data/number-with-depression-by-country.csv')
df_devInd = pd.read_csv('data/human-development-index.csv')

In [34]:
# pivot the dataframe with columns as 'year' and value as 'prevalence - depressive ...'
df_country.head()

,Entity,Code,Year,Prevalence - Depressive disorders - Sex: Both - Age: All Ages (Number)
0,Afghanistan,AFG,1990,318435.813670
1,Afghanistan,AFG,1991,329044.773956
2,Afghanistan,AFG,1992,382544.572895
3,Afghanistan,AFG,1993,440381.507393
4,Afghanistan,AFG,1994,456916.645489


In [35]:
df_country.rename(columns = {'Prevalence - Depressive disorders - Sex: Both - Age: All Ages (Number)': 'absolute_numbers'}, inplace= True)

In [36]:
df_country.sample(1)

,Entity,Code,Year,absolute_numbers
5788,Togo,TGO,2010,167476.108865


In [37]:
test = df_country.groupby(['Entity']).absolute_numbers.mean()
test

Entity
Afghanistan                   6.184937e+05
Albania                       6.628696e+04
Algeria                       1.056483e+06
American Samoa                1.339697e+03
Andean Latin America          1.128866e+06
                                  ...     
Western Sub-Saharan Africa    7.864664e+06
World                         2.157767e+08
Yemen                         6.237880e+05
Zambia                        2.765887e+05
Zimbabwe                      2.694433e+05
Name: absolute_numbers, Length: 231, dtype: float64

In [68]:
type(test)

pandas.core.series.Series

In [70]:
indexlist = [i for i in test.index]

In [39]:
len(indexlist) 

231

In [44]:
test.loc['Germany']

3495089.7958235717

In [ ]:
# pivot is probably not need for joining the tables, pivot is probably bad database design

In [82]:
df_country_pivot = df_country.pivot_table(index=['Entity'], columns=['Year'], values=['absolute_numbers'])
#df_country_pivot.loc[:,('absolute_numbers',1995)]
#df_country_pivot.loc['Germany',('absolute_numbers',1995)]
#df_country_pivot.loc['Germany',('absolute_numbers')]
df_country_pivot.loc['Germany',('absolute_numbers',(1995,2016))]


                  Year
absolute_numbers  1995    3.451053e+06
                  2016    3.638337e+06
Name: Germany, dtype: float64

In [83]:
df_country_pivot.columns

MultiIndex([('absolute_numbers', 1990),
            ('absolute_numbers', 1991),
            ('absolute_numbers', 1992),
            ('absolute_numbers', 1993),
            ('absolute_numbers', 1994),
            ('absolute_numbers', 1995),
            ('absolute_numbers', 1996),
            ('absolute_numbers', 1997),
            ('absolute_numbers', 1998),
            ('absolute_numbers', 1999),
            ('absolute_numbers', 2000),
            ('absolute_numbers', 2001),
            ('absolute_numbers', 2002),
            ('absolute_numbers', 2003),
            ('absolute_numbers', 2004),
            ('absolute_numbers', 2005),
            ('absolute_numbers', 2006),
            ('absolute_numbers', 2007),
            ('absolute_numbers', 2008),
            ('absolute_numbers', 2009),
            ('absolute_numbers', 2010),
            ('absolute_numbers', 2011),
            ('absolute_numbers', 2012),
            ('absolute_numbers', 2013),
            ('absolute_numbers', 2014),


In [84]:
df_devInd.head()

,Entity,Code,Year,Human Development Index (UNDP)
0,Afghanistan,AFG,1980,0.228
1,Afghanistan,AFG,1985,0.273
2,Afghanistan,AFG,2002,0.373
3,Afghanistan,AFG,2003,0.383
4,Afghanistan,AFG,2004,0.398


In [123]:
df_devInd_2index = df_devInd.set_index(['Entity','Year'])

In [124]:
# if not sorted, then apply:
# df_devInd_2index.sort_index(inplace = 'True')
df_devInd_2index.head(5)
# multiindex, visually beautiful, but not tidy

Code  HDI_UNDP
Entity      Year               
Afghanistan 1980  AFG     0.228
            1985  AFG     0.273
            2002  AFG     0.373
            2003  AFG     0.383
            2004  AFG     0.398

In [125]:
df_devInd_2index.reset_index()

,Entity,Year,Code,HDI_UNDP
0,Afghanistan,1980,AFG,0.228
1,Afghanistan,1985,AFG,0.273
2,Afghanistan,2002,AFG,0.373
3,Afghanistan,2003,AFG,0.383
4,Afghanistan,2004,AFG,0.398
...,...,...,...,...
4996,Zimbabwe,2013,ZWE,0.516
4997,Zimbabwe,2014,ZWE,0.525
4998,Zimbabwe,2015,ZWE,0.529
4999,Zimbabwe,2016,ZWE,0.532


In [129]:
df_devInd_2index.index

MultiIndex([('Afghanistan', 1980),
            ('Afghanistan', 1985),
            ('Afghanistan', 2002),
            ('Afghanistan', 2003),
            ('Afghanistan', 2004),
            ('Afghanistan', 2005),
            ('Afghanistan', 2006),
            ('Afghanistan', 2007),
            ('Afghanistan', 2008),
            ('Afghanistan', 2009),
            ...
            (   'Zimbabwe', 2008),
            (   'Zimbabwe', 2009),
            (   'Zimbabwe', 2010),
            (   'Zimbabwe', 2011),
            (   'Zimbabwe', 2012),
            (   'Zimbabwe', 2013),
            (   'Zimbabwe', 2014),
            (   'Zimbabwe', 2015),
            (   'Zimbabwe', 2016),
            (   'Zimbabwe', 2017)],
           names=['Entity', 'Year'], length=5001)

In [128]:
df_devInd_2index.set_index(['Entity'])

KeyError: "None of ['Entity'] are in the columns"

In [122]:
df_devInd_2index

Code  HDI_UNDP
Entity      Year               
Afghanistan 1980  AFG     0.228
            1985  AFG     0.273
            2002  AFG     0.373
            2003  AFG     0.383
            2004  AFG     0.398
...               ...       ...
Zimbabwe    2013  ZWE     0.516
            2014  ZWE     0.525
            2015  ZWE     0.529
            2016  ZWE     0.532
            2017  ZWE     0.535

[5001 rows x 2 columns]

In [85]:
df_devInd.rename(columns={'Human Development Index (UNDP)':'HDI_UNDP'},inplace =True)

In [86]:
df_devInd_pivot = df_devInd.pivot_table(index=['Entity'], columns=['Year'], values=['HDI_UNDP'])

In [91]:
df_depression_hdi = df_country_pivot.join(df_devInd_pivot)

In [95]:
df_depression_hdi.columns

MultiIndex([('absolute_numbers', 1990),
            ('absolute_numbers', 1991),
            ('absolute_numbers', 1992),
            ('absolute_numbers', 1993),
            ('absolute_numbers', 1994),
            ('absolute_numbers', 1995),
            ('absolute_numbers', 1996),
            ('absolute_numbers', 1997),
            ('absolute_numbers', 1998),
            ('absolute_numbers', 1999),
            ('absolute_numbers', 2000),
            ('absolute_numbers', 2001),
            ('absolute_numbers', 2002),
            ('absolute_numbers', 2003),
            ('absolute_numbers', 2004),
            ('absolute_numbers', 2005),
            ('absolute_numbers', 2006),
            ('absolute_numbers', 2007),
            ('absolute_numbers', 2008),
            ('absolute_numbers', 2009),
            ('absolute_numbers', 2010),
            ('absolute_numbers', 2011),
            ('absolute_numbers', 2012),
            ('absolute_numbers', 2013),
            ('absolute_numbers', 2014),


In [96]:
# if dataframe is not pivoted, or pivoted with dual index (country, year), then concat like this:
# merged_df = pd.merge(df1,df2,left_index=True,right_index=True)

In [73]:
df_devInd_pivot

Human Development Index (UNDP)                                     \
Year                                  1980   1985   1990   1991   1992   1993   
Entity                                                                          
Afghanistan                          0.228  0.273    NaN    NaN    NaN    NaN   
Albania                              0.625  0.623  0.645  0.626  0.610  0.613   
Algeria                                NaN    NaN  0.577  0.581  0.587  0.591   
Andorra                                NaN    NaN    NaN    NaN    NaN    NaN   
Angola                                 NaN    NaN    NaN    NaN    NaN    NaN   
...                                    ...    ...    ...    ...    ...    ...   
Venezuela                            0.628  0.632  0.634  0.644  0.651  0.653   
Vietnam                              0.463  0.479  0.475  0.484  0.496  0.506   
Yemen                                  NaN    NaN  0.399  0.403  0.405  0.408   
Zambia                               0.418  0.409  0.401  0.402  0.403  0.409   
Zimbabwe                             0.437  0.498  0.491  0.494  0.481  0.476   

                                         ...                              \
Year          1994   1995   1996   1997  ...   2008   2009   2010   2011   
Entity                                   ...                               
Afghanistan    NaN    NaN    NaN    NaN  ...  0.437  0.453  0.463  0.471   
Albania      0.619  0.632  0.641  0.641  ...  0.724  0.729  0.741  0.752   
Algeria      0.595  0.600  0.608  0.617  ...  0.709  0.719  0.729  0.736   
Andorra        NaN    NaN    NaN    NaN  ...  0.831  0.830  0.828  0.827   
Angola         NaN    NaN    NaN    NaN  ...  0.502  0.522  0.520  0.535   
...            ...    ...    ...    ...  ...    ...    ...    ...    ...   
Venezuela    0.654  0.658  0.660  0.664  ...  0.753  0.753  0.759  0.771   
Vietnam      0.517  0.529  0.540  0.539  ...  0.640  0.656  0.654  0.664   
Yemen        0.409  0.415  0.422  0.427  ...  0.485  0.491  0.498  0.499   
Zambia       0.408  0.412  0.416  0.419  ...  0.517  0.533  0.544  0.556   
Zimbabwe     0.473  0.467  0.466  0.459  ...  0.439  0.450  0.467  0.478   

                                                       
Year          2012   2013   2014   2015   2016   2017  
Entity                                                 
Afghanistan  0.482  0.487  0.491  0.493  0.494  0.498  
Albania      0.767  0.771  0.773  0.776  0.782  0.785  
Algeria      0.740  0.745  0.747  0.749  0.753  0.754  
Andorra      0.849  0.850  0.853  0.854  0.856  0.858  
Angola       0.543  0.554  0.564  0.572  0.577  0.581  
...            ...    ...    ...    ...    ...    ...  
Venezuela    0.774  0.776  0.778  0.775  0.766  0.761  
Vietnam      0.670  0.675  0.678  0.684  0.689  0.694  
Yemen        0.505  0.507  0.505  0.483  0.462  0.452  
Zambia       0.569  0.574  0.580  0.583  0.586  0.588  
Zimbabwe     0.505  0.516  0.525  0.529  0.532  0.535  

[190 rows x 30 columns]

In [17]:
df_edu = df_edu[['Entity', 'All levels of education (total)', 'Below upper secondary education (total)', 'Tertiary education (total)', 'Upper secondary and post-secondary non-tertiary education (total)']]

In [22]:
df_edu.head()

,Entity,All levels of education (total),Below upper secondary education (total),Tertiary education (total),Upper secondary and post-secondary non-tertiary education (total)
0,Austria,7.7,15.2,5.5,6.7
1,Belgium,7.1,11.6,4.2,7.5
2,Czech Republic,4.0,6.0,2.0,4.4
3,Denmark,8.3,15.5,6.7,8.8
4,Estonia,5.1,6.4,4.3,5.2


In [34]:
df_merged = df_edu.merge(df_gen, left_on = 'Entity', right_on = 'Country').drop(['Entity'], axis=1)

In [35]:
df_merged

,All levels of education (total),Below upper secondary education (total),Tertiary education (total),Upper secondary and post-secondary non-tertiary education (total),HDI Rank (2018),Country,1990,1991,1992,1993,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,7.7,15.2,5.5,6.7,20,Austria,0.795,0.799,0.805,0.809,...,0.886,0.895,0.897,0.899,0.896,0.904,0.906,0.909,0.912,0.914
1,7.1,11.6,4.2,7.5,17,Belgium,0.806,0.810,0.825,0.838,...,0.899,0.903,0.904,0.905,0.908,0.910,0.913,0.915,0.917,0.919
2,8.3,15.5,6.7,8.8,11,Denmark,0.799,0.804,0.809,0.815,...,0.906,0.910,0.922,0.924,0.926,0.928,0.926,0.928,0.929,0.930
3,5.1,6.4,4.3,5.2,30,Estonia,0.730,0.725,0.716,0.709,...,0.838,0.844,0.853,0.859,0.863,0.865,0.871,0.875,0.879,0.882
4,10.7,14.4,8.9,13.2,12,Finland,0.784,0.787,0.795,0.799,...,0.899,0.903,0.907,0.908,0.916,0.918,0.919,0.922,0.924,0.925
5,6.3,9.4,4.9,5.7,26,France,0.780,0.790,0.798,0.804,...,0.869,0.872,0.876,0.878,0.882,0.887,0.888,0.887,0.890,0.891
6,11.6,17.0,8.9,12.5,4,Germany,0.801,0.809,0.814,0.824,...,0.916,0.920,0.925,0.927,0.927,0.930,0.933,0.936,0.938,0.939
7,3.6,5.4,2.4,3.4,32,Greece,0.753,0.763,0.764,0.762,...,0.859,0.857,0.853,0.856,0.858,0.866,0.868,0.866,0.871,0.872
8,4.7,10.3,2.8,4.2,43,Hungary,0.704,0.702,0.706,0.720,...,0.823,0.826,0.823,0.826,0.835,0.833,0.835,0.838,0.841,0.845
9,14.4,22.5,9.5,15.7,6,Iceland,0.804,0.817,0.820,0.821,...,0.891,0.892,0.901,0.908,0.920,0.924,0.927,0.932,0.935,0.938
